In [1]:
import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import os
import imageio
from mapFunctions import getMap

# --- Paths ---
monthly_base = '/data/user/fmcnally/anisotropy/maps/merged_monthly/IC86_N10_sid_{:02d}.fits'
all_map_file = '/data/user/fmcnally/anisotropy/maps/merged/IC86_N10_sid.fits'
output_dir = './solar_dipole_frames'
gif_path = 'solar_dipole.gif'

# Create output directory if needed
os.makedirs(output_dir, exist_ok=True)

# --- Load 12-year map ---
if not os.path.isfile(all_map_file):
    raise FileNotFoundError(f"All-data map not found: {all_map_file}")
all_data, all_bg, _ = hp.read_map(all_map_file, field=(0, 1, 2))
all_relint = getMap([all_map_file], mapName='relint', smooth=20, verbose=False, mask=True)

# --- Loop over months ---
frame_files = []

for month in range(1, 13):
    month_file = monthly_base.format(month)
    if not os.path.isfile(month_file):
        print(f"Skipping month {month:02d}: file not found")
        continue

    try:
        month_data, month_bg, _ = hp.read_map(month_file, field=(0, 1, 2))
        month_relint = getMap([month_file], mapName='relint', smooth=20, verbose=False, mask=True)
    except Exception as e:
        print(f"Error loading month {month:02d}: {e}")
        continue

    if month_relint is None or all_relint is None:
        print(f"Skipping month {month:02d}: map could not be loaded.")
        continue

    # --- Compute difference map ---
    diff_map = month_relint - all_relint

    # --- Mask northern hemisphere and invalid pixels ---
    nside = hp.get_nside(diff_map)
    npix = hp.nside2npix(nside)
    theta, _ = hp.pix2ang(nside, np.arange(npix))

    diff_map_masked = diff_map.copy()
    north_mask = theta < (np.pi / 2)
    invalid_mask = (~np.isfinite(diff_map)) | (diff_map == 0) | (diff_map == hp.UNSEEN)
    full_mask = north_mask | invalid_mask
    diff_map_masked[full_mask] = hp.UNSEEN

    # --- Plot and save map ---
    fig = plt.figure(figsize=(8, 6))
    hp.mollview(
        diff_map_masked,
        title=f'Solar Dipole Map - Month {month:02d}',
        unit='Relative Intensity',
        norm='hist',
        notext=True,
        cmap='coolwarm',
        badcolor='lightgrey',
        fig=1
    )
    hp.graticule(visible=False)
    plt.axis('off')

    out_png = os.path.join(output_dir, f'dipole_month_{month:02d}.png')
    plt.savefig(out_png, dpi=200, bbox_inches='tight')
    plt.close()
    frame_files.append(out_png)

# --- Create GIF ---
print("Creating GIF...")
images = [imageio.v2.imread(f) for f in frame_files]
imageio.mimsave(gif_path, images, duration=0.8)  # Adjust duration as needed

print(f"Saved GIF: {gif_path}")


Creating GIF...
Saved GIF: solar_dipole.gif


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from scipy.optimize import curve_fit
from astropy.time import Time
from astropy.coordinates import get_body_barycentric_posvel
from astropy import units as u, constants as const
from mapFunctions import getMap
import os
import random

# --- Select a Random Year ---
year = random.randint(2000, 2024)
print(f"Selected Year: {year}")
months = np.arange(1, 13)

# --- Time array: all days of each month ---
t_daily = []
for m in months:
    # Determine number of days in month (handles leap years via Time)
    t_start = Time(f"{year}-{m:02d}-01T00:00:00", scale='utc')
    t_end = Time(f"{year}-{m%12 + 1:02d}-01T00:00:00", scale='utc') if m < 12 else Time(f"{year+1}-01-01T00:00:00", scale='utc')
    n_days = int((t_end - t_start).value)
    t_daily += [f"{year}-{m:02d}-{d+1:02d}T00:00:00" for d in range(n_days)]

t_array = Time(t_daily, scale='utc')

# --- Get Earth's barycentric velocity (ICRS) for each day ---
_, vel_array = get_body_barycentric_posvel('earth', t_array)

# Split daily velocities into monthly blocks
vx_daily = vel_array.x.to_value(u.km/u.s)
vy_daily = vel_array.y.to_value(u.km/u.s)
vz_daily = vel_array.z.to_value(u.km/u.s)
speed_daily = vel_array.norm().to(u.km/u.s).value

month_day_counts = [len([t for t in t_daily if int(t[5:7]) == m]) for m in months]
idx_split = np.cumsum(month_day_counts)

vx_split = np.split(vx_daily, idx_split[:-1])
vy_split = np.split(vy_daily, idx_split[:-1])
vz_split = np.split(vz_daily, idx_split[:-1])
speed_split = np.split(speed_daily, idx_split[:-1])

# --- CG Amplitude & Phase (Compton-Getting formula) ---
gamma = 2.6
cg_factor = gamma + 2
cg_phases = []
cg_amplitudes = []

for i in range(12):
    vx_mean = np.mean(vx_split[i])
    vy_mean = np.mean(vy_split[i])
    vz_mean = np.mean(vz_split[i])
    v = np.array([vx_mean, vy_mean, vz_mean])
    v_hat = v / np.linalg.norm(v)
    
    # Calculate RA (phase) from unit vector
    x, y, z = v_hat
    ra = np.arctan2(y, x)
    if ra < 0:
        ra += 2 * np.pi
    cg_phases.append(np.degrees(ra))

    # Amplitude calculation
    v_mag = np.mean(speed_split[i])  # average speed for the month
    amp = cg_factor * (v_mag * 1e3 / const.c.value)  # unitless (ratio v/c)
    cg_amplitudes.append(amp)

# --- Load full year map (all months combined) ---
monthly_base = '/data/user/fmcnally/anisotropy/maps/merged_monthly/IC86_N10_sid_{:02d}.fits'
all_map_file = '/data/user/fmcnally/anisotropy/maps/merged/IC86_N10_sid.fits'

all_data, all_bg, _ = hp.read_map(all_map_file, field=(0, 1, 2))
all_relint = getMap([all_map_file], mapName='relint', smooth=0, verbose=False, mask=True)

# --- Cosine fit model ---
def cos2d(ang, A, phi):
    psi = hp.rotator.angdist(ang, [0.5 * np.pi, phi])
    return A * np.cos(psi)

# --- Fit each month ---
for month in range(1, 13):
    A_CG = cg_amplitudes[month - 1]
    phi_CG = cg_phases[month - 1]
    print(f"[Month {month:02d}] CG: A = {A_CG:.3e}, φ = {phi_CG:.2f}°")

    month_file = monthly_base.format(month)
    if not os.path.isfile(month_file):
        print(f"  Skipping: File not found.")
        continue

    month_data, month_bg, _ = hp.read_map(month_file, field=(0, 1, 2))
    month_relint = getMap([month_file], mapName='relint', smooth=0, verbose=False, mask=True)
    if month_relint is None:
        print(f"  Skipping: Map could not be loaded.")
        continue

    # --- Differential map ---
    diff_map = month_relint - all_relint
    nside = hp.get_nside(diff_map)
    theta, phi = hp.pix2ang(nside, np.arange(hp.nside2npix(nside)))
    diff_map_masked = diff_map.copy()
    diff_map_masked[theta < (np.pi/2)] = hp.UNSEEN

    # --- Select field-of-view pixels (90° to 120° from zenith) ---
    fov_pixels = np.where((theta >= np.radians(90)) & (theta <= np.radians(120)))[0]
    fov_angles = hp.pix2ang(nside, fov_pixels)
    fov_values = diff_map[fov_pixels]

    # --- Error propagation ---
    sigma_month = np.sqrt((month_data / month_bg)**2 * (1.0 / month_data + 1.0 / (month_bg * 20)))
    sigma_all   = np.sqrt((all_data / all_bg)**2 * (1.0 / all_data + 1.0 / (all_bg * 20)))
    sigma_fov   = np.sqrt(sigma_month[fov_pixels]**2 + sigma_all[fov_pixels]**2)

    try:
        popt, pcov = curve_fit(
            cos2d, fov_angles, fov_values,
            sigma=sigma_fov,
            bounds=([0, 0], [0.01, 2 * np.pi])
        )
        A_fit, phi_fit = popt
        A_err, phi_err = np.sqrt(np.diag(pcov))
        print(f"  Data Fit: A = {A_fit:.4g} ± {A_err:.1g}, φ = {np.degrees(phi_fit):.2f}° ± {np.degrees(phi_err):.2f}°")
    except Exception as e:
        print(f"  Fit failed: {e}")
        continue
predicted_amplitudes = np.array(cg_amplitudes)  # or your fitted amplitudes if preferred
predicted_phases = np.array(cg_phases)

Selected Year: 2000
[Month 01] CG: A = 4.644e-04, φ = 203.14°
  Data Fit: A = 0.000153 ± 9e-06, φ = 201.67° ± 3.22°
[Month 02] CG: A = 4.627e-04, φ = 232.63°
  Data Fit: A = 0.000137 ± 9e-06, φ = 238.24° ± 3.72°
[Month 03] CG: A = 4.595e-04, φ = 264.33°
  Data Fit: A = 0.0001478 ± 9e-06, φ = 269.00° ± 3.36°
[Month 04] CG: A = 4.557e-04, φ = 296.80°
  Data Fit: A = 0.0001312 ± 9e-06, φ = 299.84° ± 3.92°
[Month 05] CG: A = 4.523e-04, φ = 326.99°
  Data Fit: A = 0.0001745 ± 9e-06, φ = 327.30° ± 2.91°
[Month 06] CG: A = 4.501e-04, φ = 354.81°
  Data Fit: A = 0.0001668 ± 9e-06, φ = 355.09° ± 3.06°
[Month 07] CG: A = 4.498e-04, φ = 22.16°
  Data Fit: A = 0.0001677 ± 9e-06, φ = 20.04° ± 3.07°
[Month 08] CG: A = 4.514e-04, φ = 51.68°
  Data Fit: A = 0.0001209 ± 9e-06, φ = 51.27° ± 4.32°
[Month 09] CG: A = 4.544e-04, φ = 83.30°
  Data Fit: A = 0.0001714 ± 9e-06, φ = 78.28° ± 3.03°
[Month 10] CG: A = 4.583e-04, φ = 115.77°
  Data Fit: A = 0.0001565 ± 8e-06, φ = 116.78° ± 3.06°
[Month 11] CG: A =